In [1]:
%matplotlib inline

### os
import os
import sys

### datetimes
from datetime import datetime, timedelta

### scipy
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import cartopy.crs as ccrs
import dask
from dask.diagnostics import ProgressBar
from tqdm import tqdm

### plotting
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns


In [2]:
import pathlib

HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd()


In [3]:
import sys

In [4]:
sys.path.append('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/')

In [5]:
from ICU_Water_Watch.plot import map_precip_anoms
from ICU_Water_Watch.GPM import calc_anoms_and_pctscores

In [6]:
year = 2021

In [7]:
ndays = 90

### paths 

In [8]:
dpath = HOME.joinpath('operational/ICU/ops/data/GPM_IMERG/daily/extended_SP')

In [9]:
clim_file = f"GPM_IMERG_daily.v06.2001.2020_precipitationCal_{ndays}d_runsum.nc"

In [10]:
clim = xr.open_dataset(dpath.joinpath(clim_file))

In [11]:
clim.time.dt.dayofyear

<xarray.DataArray 'dayofyear' (time: 7216)>
array([ 90,  91,  92, ..., 364, 365, 366])
Coordinates:
  * time     (time) datetime64[ns] 2001-03-31 2001-04-01 ... 2020-12-31

### generate the list of files to read prior to the calculation of the running accumulations

In [12]:
first_date = datetime(year, 1, 1)

In [13]:
first_date = first_date - timedelta(days=ndays)

In [14]:
last_date = datetime(year, 12, 31)

In [15]:
ldates = pd.date_range(start=first_date, end=last_date)

In [16]:
lfiles = [dpath.joinpath(f"GPM_IMERG_daily.v06.{d:%Y.%m.%d}.nc") for d in ldates]

In [17]:
dset = xr.open_mfdataset(lfiles)

### rechunk 

In [18]:
dset = dset.chunk({"time":-1, "lat":10, "lon":10})

### calculates the running accumulations 

In [19]:
dset = dset.rolling({'time':ndays}).sum('time')

In [20]:
with ProgressBar(): 
    dset = dset.compute()

[########################################] | 100% Completed | 26.5s


In [21]:
dset = dset.isel(time=slice(ndays, None))

### calculates the anomalies and the percentiles of scores 

In [22]:
thresholds = [0, 5, 10, 25, 90.01, 100]

rgbs = ['#F04E37', '#F99D1C','#FFDE40','#FFFFFF', '#33BBED']

ticks_marks = np.diff(np.array(thresholds)) / 2.

ticks = [thresholds[i] + ticks_marks[i] for i in range(len(thresholds) - 1)]

cbar_ticklabels = ["Severely dry (< 5%)",'Seriously dry (< 10%)', 'Warning (< 25%)', 'Near or Wetter', 'Seriously wet (> 90%)']

# arguments for the colorbar 

cbar_kwargs={'shrink':0.5, 'pad':0.01, 'extend':'neither', 'drawedges':True, 'ticks':ticks, 'aspect':15}

cmap = matplotlib.colors.ListedColormap(rgbs, name='EAR Watch')

In [23]:
window_clim = 2

In [24]:
dset_interp = {}
dset_interp['lat'] = (('lat'), np.linspace(-49.875, 24.875, 300, endpoint=True))
dset_interp['lon'] = (('lon'), np.linspace(125.125, 239.875, 300, endpoint=True))
dset_interp = xr.Dataset(dset_interp)

In [25]:
for d in dset.time.data: 
    
    dset_sub = dset.sel(time=d).expand_dims({'time':[d]})

    dd = [d + timedelta(days=x) for x in range(-window_clim, window_clim+1)]
    
    clim_sub = []

    for date in dd: 
    
        c = clim.sel(time=((clim.time.dt.month == date.month) & (clim.time.dt.day == date.day)))
    
        clim_sub.append(c)
    
    clim_sub = xr.concat(clim_sub, dim='time')
    
    clim_sub = clim_sub.sortby('time')
    
    anoms_pctscore = calc_anoms_and_pctscores(dset_sub, clim_sub) 
    
    dataarray = anoms_pctscore['pctscore'].squeeze()
    
    dataarray = dataarray.interp_like(dset_interp)
    
    f, ax = plt.subplots(figsize=(13, 8), subplot_kw={'projection':ccrs.PlateCarree(central_longitude=180)})

    im = dataarray.plot.contourf(ax=ax, levels=thresholds, cmap=cmap, transform=ccrs.PlateCarree(), add_colorbar=False)

    cbar_ax = ax.axes.inset_axes([1.01, 0.6, 0.025, 0.38] , zorder=1)

    # plots the colorbar in these axes 

    cb = plt.colorbar(im, cax=cbar_ax, **cbar_kwargs)

    # removes the minor ticks 

    cb.ax.minorticks_off() 

    # plots the ticklabels 

    cbar_ax.set_yticklabels(cbar_ticklabels)

    ax.coastlines(resolution='10m')

    ax.set_title(f"{ndays} days accumulation ending {d:%Y-%m-%d}")
    
    f.savefig(f"./tmp/2021_EAR_{ndays}days_{d.dayofyr:03}.png", dpi=200, bbox_inches='tight', facecolor='w') 
    
    plt.close(f)
    

/home/nicolasf/mambaforge/envs/climetlab/lib/python3.8/site-packages/cartopy/crs.py:825: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
/home/nicolasf/mambaforge/envs/climetlab/lib/python3.8/site-packages/cartopy/crs.py:877: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
/home/nicolasf/mambaforge/envs/climetlab/lib/python3.8/site-packages/cartopy/crs.py:944: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
/home/nicolasf/mamb